In [11]:
import numpy as np
import pandas as pd

# Plotting using matplotlib
import matplotlib.pyplot as plt
%matplotlib widget

# Plotting using plotly
import plotly.graph_objects as go

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import display, clear_output

In [12]:
# Rocket tracking setup parameters
### Coordinates of rocket (m)(Radio receiver)
rocketStartX, rocketStartY, rocketStartZ = 0, 0, 1.8

### Coordinates of azimuth center of rotation (m)
azimuthMotorX, azimuthMotorY, azimuthMotorZ = 600, 0, 0

### Coordinates of elevation center of rotation (m)
elevationMotorX, elevationMotorY, elevationMotorZ = 400, 400, 0

### Home angles 
homeAzimuth, homeElevation = 0, 0

In [13]:
# Column trimmer (CSV has more data columns than headers)
with open('test_flight.csv', 'r') as f:
    firstLine = f.readline()
    secondLine = f.readline()

# Count the number of commas in the header and data lines
numHeadColumns = firstLine.count(',') + 1
numDataColumns = secondLine.count(',') + 1

# Trim the columns to use with the minimum of the two counts
trimColumns = min(numHeadColumns, numDataColumns)
columnsToUse = list(range(trimColumns))

# Read CSV with proper column data
testCSV = pd.read_csv('test_flight.csv', usecols=columnsToUse)

# Offset the positions with the first value and the initial rocket coordinates
testCSV.iloc[:, 1] = testCSV.iloc[:, 1] - testCSV.iloc[0, 1] + rocketStartX
testCSV.iloc[:, 2] = testCSV.iloc[:, 2] - testCSV.iloc[0, 2] + rocketStartY
testCSV.iloc[:, 3] = testCSV.iloc[:, 3] - testCSV.iloc[0, 3] + rocketStartZ

testCSV.head()


,# Time (s),X (m),Y (m),Z (m),E0,E1,E2,E3,W1 (rad/s),W2 (rad/s),W3 (rad/s)
0,0.000000,0.0,0.0,1.800000,0.0,0.000000,0.000000,0.999048,-0.043619,-0.0,0.0
1,0.001590,0.0,0.0,1.800000,0.0,0.000000,0.000000,0.999048,-0.043619,0.0,0.0
2,0.003179,0.0,0.0,1.800000,0.0,0.000000,0.000000,0.999048,-0.043619,0.0,0.0
3,0.003815,0.0,0.0,1.800000,0.0,0.000059,0.000679,0.999048,-0.043619,0.0,0.0
4,0.004451,0.0,0.0,1.800002,0.0,0.000219,0.002503,0.999048,-0.043619,0.0,0.0


In [14]:
# Calculate azimuth angle between azimuthMotor and rocket
def calculateAzimuthAngle(x, y):
    return np.arctan2(y - azimuthMotorY, x - azimuthMotorX)

# Calculate elevation angle between elevationMotor (at a different height from azimuthMotor) and rocket
def calculateElevationAngle(x, y, z):
    return np.arctan2(z - elevationMotorZ, np.sqrt((x - elevationMotorX)**2 + (y - elevationMotorY)**2))

# Combine both angles into a single unit vector
def calculateUnitVector(azimuth, elevation):
    # Using anti-clockwise azimuth and elevation angles
    return np.array([np.cos(azimuth)*np.cos(elevation), np.sin(azimuth)*np.cos(elevation), np.sin(elevation)])

# Helper function for interpolating between two single coordinates and time values
def interpolateCoordinates(x1, y1, z1, x2, y2, z2, t1, t2, t):
    return x1 + (x2 - x1) * (t - t1) / (t2 - t1), y1 + (y2 - y1) * (t - t1) / (t2 - t1), z1 + (z2 - z1) * (t - t1) / (t2 - t1)

In [15]:
# Create range of time
initialTime = testCSV.iloc[0, 0]
finalTime = testCSV.iloc[-1, 0]
steps = 100
timeRange = np.linspace(initialTime, finalTime, steps)

# Create a simulation dataframe for plotting positions, angles and vectors
simList = []

# Interpolate between the data points to create a smooth simulation
for time in timeRange:
    
    # Find the two closest points in time
    before = testCSV[testCSV.iloc[:,0] <= time].iloc[-1]
    if testCSV[testCSV.iloc[:,0] > time].shape[0] > 0:
        after = testCSV[testCSV.iloc[:,0] > time].iloc[0]
        
        # Interpolate between the two points
        x, y, z = interpolateCoordinates(before.iloc[1], before.iloc[2], 
                                         before.iloc[3], after.iloc[1], 
                                         after.iloc[2], after.iloc[3], 
                                         before.iloc[0], after.iloc[0], time)
        azimuth = calculateAzimuthAngle(x, y)
        elevation = calculateElevationAngle(x, y, z)
        unitVector = calculateUnitVector(azimuth, elevation)
        
        # Add entry to the simulation dataframe
        simList.append([time, x, y, z, azimuth, elevation, 
                        unitVector[0], unitVector[1], unitVector[2]]) 

simulation = pd.DataFrame(simList, columns=['Time', 'X', 'Y', 'Z', 'Azimuth',
                                            'Elevation', 'UnitVectorX', 
                                            'UnitVectorY', 'UnitVectorZ'])

In [16]:
# Plotly
# Create a trace for the rocket path
trace = go.Scatter3d(
    x=simulation['X'],
    y=simulation['Y'],
    z=simulation['Z'],
    mode='markers',
    marker=dict(
        size=4,
        color=simulation['Time'],  # set color to an array/list of desired values
        colorscale='Viridis',  # choose a colorscale
        opacity=0.8
    ),
    text=simulation['Time'].map('{:.3f}'.format) + ' s'  # this line sets the hover text
)

data = [trace]

# Create layout for the plot
layout = go.Layout(
    title='Rocket Path',
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z'),
    ),
    showlegend=False,
    autosize=False,
    width=700,
    height=600,
)

figStatic = go.Figure(data=data, layout=layout)

# Plot coordinates of antenna rig
figStatic.add_trace(go.Scatter3d(x=[elevationMotorX], y=[elevationMotorY], z=[elevationMotorZ],
                                 mode='markers', marker=dict(size=5, color='red'), name='Antenna rig'))

# Change limits of the plot
figStatic.update_layout(scene=dict(aspectmode='cube', 
                                   xaxis=dict(range=[-2000, 2000]), 
                                   yaxis=dict(range=[-2000, 2000]), 
                                   zaxis=dict(range=[0, 4000])))

figStatic.show()

In [17]:
# Create a trace for the unit vector
vectorMagnitude = 500

unit_vector_trace = go.Scatter3d(
    x=simulation['UnitVectorX'] * vectorMagnitude + elevationMotorX,
    y=simulation['UnitVectorY'] * vectorMagnitude + elevationMotorY,
    z=simulation['UnitVectorZ'] * vectorMagnitude + elevationMotorZ,
    mode='lines',
    line=dict(
        width=2,
        color='red',
    ),
    name='Unit Vector'
)

frames = [go.Frame(data=[go.Scatter3d(
    x=simulation['X'][:i],
    y=simulation['Y'][:i],
    z=simulation['Z'][:i],
    mode='markers',
    marker=dict(
        size=4,
        color=simulation['Time'][:i],
        colorscale='Viridis',
        opacity=0.8
    ),
    text=simulation['Time'][:i].map('{:.3f}'.format) + ' s'
),
go.Scatter3d(
    x=simulation['UnitVectorX'][:i] * vectorMagnitude + elevationMotorX,
    y=simulation['UnitVectorY'][:i] * vectorMagnitude + elevationMotorY,
    z=simulation['UnitVectorZ'][:i] * vectorMagnitude + elevationMotorZ,
    mode='lines',
    line=dict(
        width=2,
        color='red',
    ),
    name='Unit Vector'
)]) for i in range(1, len(simulation) + 1)]

layout = go.Layout(
    title='Rocket Path (unit vector currently not correct - center of scale is off)',
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z'),
    ),
    showlegend=False,
    updatemenus=[dict(type="buttons",
                        showactive=False,
                        buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True, "transition": {"duration": 300}}]),
                                dict(label="Stop",
                                        method="animate",
                                        args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}])])],
    # Set the duration of the animation
    sliders=[dict(steps=[dict(method='animate',
                              args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True)]),
                         dict(label='Time',
                              method='animate',
                              args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True)])])],
)

# Create animation figure
figSim = go.Figure(data=[trace, unit_vector_trace], frames=frames, layout=layout)

# Plot coordinates of antenna rig
figSim.add_trace(go.Scatter3d(x=[elevationMotorX], y=[elevationMotorY], z=[elevationMotorZ],
                                 mode='markers', marker=dict(size=5, color='red'), name='Antenna rig'))

# Change limits of the plot
figSim.update_layout(scene=dict(aspectmode='cube', 
                                   xaxis=dict(range=[-2000, 2000]), 
                                   yaxis=dict(range=[-2000, 2000]), 
                                   zaxis=dict(range=[0, 4000])))

figSim.show()

In [44]:
# Plotly animation
# Create a trace for the rocket path
trace = go.Scatter3d(
    x=simulation['X'],
    y=simulation['Y'],
    z=simulation['Z'],
    mode='markers',
    marker=dict(
        size=4,
        color=simulation['Time'],  # set color to an array/list of desired values
        colorscale='Viridis',  # choose a colorscale
        opacity=0.8
    ),
    text=simulation['Time'].map('{:.3f}'.format) + ' s'  # this line sets the hover text
)

frames = [go.Frame(data=[go.Scatter3d(
    x=simulation['X'][:i],
    y=simulation['Y'][:i],
    z=simulation['Z'][:i],
    mode='markers',
    marker=dict(
        size=4,
        color=simulation['Time'][:i],
        colorscale='Viridis',
        opacity=0.8
    ),
    text=simulation['Time'][:i].map('{:.3f}'.format) + ' s'
)]) for i in range(1, len(simulation) + 1)]

# Create layout for the plot
layout = go.Layout(
    title='Rocket Path',
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z'),
    ),
    showlegend=False,
    updatemenus=[dict(type="buttons",
                        showactive=False,
                        buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True, "transition": {"duration": 300}}]),
                                dict(label="Stop",
                                        method="animate",
                                        args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}])])],
    # Set the duration of the animation
    sliders=[dict(steps=[dict(method='animate',
                              args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True)]),
                         dict(label='Time',
                              method='animate',
                              args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True)])])],
)

# Create animation figure
figSim = go.Figure(data=[trace], frames=frames, layout=layout)

# Plot coordinates of antenna rig
figSim.add_trace(go.Scatter3d(x=[elevationMotorX], y=[elevationMotorY], z=[elevationMotorZ],
                                 mode='markers', marker=dict(size=5, color='red'), name='Antenna rig'))

# Change limits of the plot
figSim.update_layout(scene=dict(aspectmode='cube', 
                                   xaxis=dict(range=[-2000, 2000]), 
                                   yaxis=dict(range=[-2000, 2000]), 
                                   zaxis=dict(range=[0, 4000])))

figSim.show()